In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

from kaggle.competitions import twosigmanews

['marketdata_sample.csv', 'news_sample.csv']


In [2]:
env = twosigmanews.make_env()
(marketdf, newsdf) = env.get_training_data()

Loading the data... This could take a minute.
Done!


In [3]:
def prepare_data(marketdf, newsdf):

    # a bit of feature engineering
    marketdf['time'] = marketdf.time.dt.strftime("%Y%m%d").astype(int)
    newsdf['time']   = newsdf.time.dt.strftime("%Y%m%d").astype(int)
    
    # filter pre-2012 data, no particular reason
    marketdf = marketdf.loc[marketdf['time'] > 20120000]
    newsdf   = newsdf.loc[newsdf['time'] > 20120000]
    
    marketdf['today_trend'] = marketdf['close'] / marketdf['open']
    marketdf['middle']      = (marketdf['close'] + marketdf['open'])/2
    marketdf['money']       = marketdf['volume'] * marketdf['middle']
    
    marketdf['returnsClosePrevRaw_sum'] = marketdf['returnsClosePrevRaw1'] + marketdf['returnsClosePrevRaw10']
    marketdf['returnsOpenPrevRaw_sum']  = marketdf['returnsOpenPrevRaw1'] + marketdf['returnsOpenPrevRaw10']
    
    marketdf['returnsClosePrevRaw_avg'] = (marketdf['returnsClosePrevRaw1'] + marketdf['returnsClosePrevRaw10']) / 2
    marketdf['returnsOpenPrevRaw_avg']  = (marketdf['returnsOpenPrevRaw1'] + marketdf['returnsOpenPrevRaw10']) / 2
    
    marketdf['returnsCloseOpenRaw1_sum']  = marketdf['returnsClosePrevRaw1'] + marketdf['returnsOpenPrevRaw1']
    marketdf['returnsCloseOpenRaw10_sum'] = marketdf['returnsClosePrevRaw10'] + marketdf['returnsOpenPrevRaw10']
    
    marketdf['returnsCloseOpenRaw1_avg']  = (marketdf['returnsClosePrevRaw1'] + marketdf['returnsOpenPrevRaw1']) / 2
    marketdf['returnsCloseOpenRaw10_avg'] = (marketdf['returnsClosePrevRaw10'] + marketdf['returnsOpenPrevRaw10']) / 2
    
    marketdf['returnsClosePrevMktres_sum'] = marketdf['returnsClosePrevMktres1'] + marketdf['returnsClosePrevMktres10']
    marketdf['returnsOpenPrevMktres_sum']  = marketdf['returnsOpenPrevMktres1'] + marketdf['returnsOpenPrevMktres10']
    
    marketdf['returnsClosePrevMktres_avg'] = (marketdf['returnsClosePrevMktres1'] + marketdf['returnsClosePrevMktres10']) / 2
    marketdf['returnsOpenPrevMktres_avg']  = (marketdf['returnsOpenPrevMktres1'] + marketdf['returnsOpenPrevMktres10']) / 2
    
    marketdf['returnsCloseOpenMktres1_sum']  = marketdf['returnsClosePrevMktres1'] + marketdf['returnsOpenPrevMktres1']
    marketdf['returnsCloseOpenMktres10_sum'] = marketdf['returnsClosePrevMktres10'] + marketdf['returnsOpenPrevMktres10']
    
    marketdf['returnsCloseOpenMktres1_avg']  = (marketdf['returnsClosePrevMktres1'] + marketdf['returnsOpenPrevMktres1']) / 2
    marketdf['returnsCloseOpenMktres10_avg'] = (marketdf['returnsClosePrevMktres10'] + marketdf['returnsOpenPrevMktres10']) / 2
    
    marketdf.drop(['open', 'close', 'middle'], axis = 1, inplace = True)
    
    newsdf.loc[newsdf['firstMentionSentence'] == 0, 'firstMentionSentence'] = newsdf.loc[newsdf['firstMentionSentence'] == 0, 'sentenceCount']
    
    newsdf['assetCode'] = newsdf['assetCodes'].map(lambda x: list(eval(x))[0])
    newsdf['position'] = newsdf['firstMentionSentence'] / newsdf['sentenceCount']
    newsdf['coverage'] = newsdf['sentimentWordCount'] / newsdf['wordCount']
    
    # get rid of extra junk from news data
    droplist = ['sourceTimestamp','firstCreated','sourceId','headline','takeSequence','provider','firstMentionSentence'
                #'sentenceCount'
                ,'bodySize','headlineTag','marketCommentary','subjects','audiences'
                #,'sentimentClass'
                ,'assetName', 'assetCodes'
                #,'urgency'
                ,'wordCount','sentimentWordCount']
    newsdf.drop(droplist, axis=1, inplace=True)
    marketdf.drop(['assetName', 'volume'], axis=1, inplace=True)
    
    # combine multiple news reports for same assets on same day
    newsgp = newsdf.groupby(['time','assetCode'], sort=False).agg(['min', 'max', 'mean', 'std'])
    newsgp.columns = pd.Index(["{}_{}".format(e[0], e[1]) for e in newsgp.columns.tolist()])
    
    # join news reports to market data, note many assets will have many days without news data
    return pd.merge(marketdf, newsgp, how='left', on=['time', 'assetCode'], copy=False) #, right_on=['time', 'assetCodes'])

In [4]:
print('preparing data...')

cdf = prepare_data(marketdf, newsdf)    
del marketdf, newsdf

preparing data...


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [ ]:
cdf.head()

In [5]:
from sklearn.model_selection import train_test_split
import gc

print('building training set...')

cdf['target'] = (cdf['returnsOpenNextMktres10'] > 0.0).astype(int)
features = [col for col in cdf.columns if col not in ['time', 'assetCode', 'universe', 'returnsOpenNextMktres10', 'target']]

#######################################################
##
## LightGBM
##
#######################################################
import lightgbm as lgb
print ('Training lightgbm')

params = {"objective" : "binary",
          "metric" : "binary_logloss",
          "num_leaves" : 60,
          "max_depth": 10,
          "learning_rate" : 0.005,
          "bagging_fraction" : 0.9,  # subsample
          "feature_fraction" : 0.9,  # colsample_bytree
          "bagging_freq" : 5,        # subsample_freq
          "bagging_seed" : 2018,
          "lambda_l1" : 1.0,
          "verbosity" : -1 }

print("using params ", params)
print("using features ", features)
print("")

dates = list(cdf['time'].unique())

k_fold = True

if k_fold :
    K = 1
    for k in range(1, K + 1) :

        k_dates = dates[:int(len(dates)*k/(K))]

        train_idx, val_idx = train_test_split(k_dates, test_size = 0.15, shuffle = False)
        #print(train_idx)
        #print(val_idx)

        train_idx = cdf[cdf['time'].isin(train_idx)].index.values
        val_idx = cdf[cdf['time'].isin(val_idx)].index.values

        print(k, "K using data from ", cdf.loc[train_idx, 'time'].min(), " to ", cdf.loc[val_idx, 'time'].max())

        train_data = lgb.Dataset(cdf.loc[train_idx, features], cdf.loc[train_idx, 'target'])
        val_data   = lgb.Dataset(cdf.loc[val_idx, features], cdf.loc[val_idx, 'target'])

        lgb.reset_parameter()
        model = lgb.train(params, train_data, 10000, valid_sets = [train_data, val_data], early_stopping_rounds = 100, verbose_eval = 50)
        
        feature_imp = pd.DataFrame(sorted(zip(features, model.feature_importance(), model.feature_importance("gain"))), columns=['feature', 'split', 'gain']).sort_values(by=['split'])
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            print(feature_imp)
        
        predict = model.predict(cdf.loc[val_idx, features], num_iteration = model.best_iteration)
        confidence_valid = predict.reshape(-1) * 2 - 1
        x_t_i = confidence_valid * cdf.loc[val_idx, 'returnsOpenNextMktres10'] * cdf.loc[val_idx, 'universe']
        data = {'day' : cdf.loc[val_idx, 'time'], 'x_t_i' : x_t_i}
        df_predict = pd.DataFrame(data)
        x_t = df_predict.groupby('day').sum().values.flatten()
        mean = np.mean(x_t)
        std = np.std(x_t)
        score_valid = mean / std
        print("two sigma score : ", score_valid)
        print("")

        gc.collect()
else :
    pass

#lgtrain, lgval = lgb.Dataset(Xt, Yt[:,0]), lgb.Dataset(Xv, Yv[:,0])
#lgbmodel = lgb.train(params, lgtrain, 2000, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=200)

building training set...
Training lightgbm
using params  {'objective': 'binary', 'metric': 'binary_logloss', 'num_leaves': 60, 'max_depth': 10, 'learning_rate': 0.005, 'bagging_fraction': 0.9, 'feature_fraction': 0.9, 'bagging_freq': 5, 'bagging_seed': 2018, 'lambda_l1': 1.0, 'verbosity': -1}
using features  ['returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'returnsClosePrevRaw10', 'returnsOpenPrevRaw10', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10', 'today_trend', 'money', 'returnsClosePrevRaw_sum', 'returnsOpenPrevRaw_sum', 'returnsClosePrevRaw_avg', 'returnsOpenPrevRaw_avg', 'returnsCloseOpenRaw1_sum', 'returnsCloseOpenRaw10_sum', 'returnsCloseOpenRaw1_avg', 'returnsCloseOpenRaw10_avg', 'returnsClosePrevMktres_sum', 'returnsOpenPrevMktres_sum', 'returnsClosePrevMktres_avg', 'returnsOpenPrevMktres_avg', 'returnsCloseOpenMktres1_sum', 'returnsCloseOpenMktres10_sum', 'returnsCloseOpenMktres1_avg', 'returnsCloseOpenMktres10_avg

two sigma score :  0.3870538325173211



In [ ]:
cdf.columns

In [ ]:
"""
Early stopping, best iteration is:
[223]	training's binary_logloss: 0.685427	valid_1's binary_logloss: 0.689652
two sigma score :  0.38887642144890827
generating predictions...
"""

In [ ]:
"""
params = {"objective" : "binary",
          "metric" : "binary_logloss",
          "num_leaves" : 60,
          "max_depth": -1,
          "learning_rate" : 0.005,
          "bagging_fraction" : 0.9,  # subsample
          "feature_fraction" : 0.9,  # colsample_bytree
          "bagging_freq" : 5,        # subsample_freq
          "bagging_seed" : 2018,
          "verbosity" : -1 }

Early stopping, best iteration is:
[219]	training's binary_logloss: 0.685482	valid_1's binary_logloss: 0.689655
    two sigma score :  0.3885975760360274
"""

In [9]:
##### scaling
def post_scaling(df):
    mean, std = np.mean(df), np.std(df)
    df = (df - mean)/ (std * 8)
    return np.clip(df,-1,1)
############################################################
print("generating predictions...")
preddays = env.get_prediction_days()
for marketdf, newsdf, predtemplatedf in preddays:
    cdf = prepare_data(marketdf, newsdf)
    print("predict ", cdf['time'].max())
    Xp = cdf[features].values
    preds = model.predict(Xp, num_iteration = model.best_iteration) * 2 - 1
    predsdf = pd.DataFrame({'ast':cdf['assetCode'],'conf':post_scaling(preds)})
    predtemplatedf['confidenceValue'][predtemplatedf['assetCode'].isin(predsdf.ast)] = predsdf['conf'].values
    env.predict(predtemplatedf)
    gc.collect()

env.write_submission_file()

generating predictions...


Exception: You can only call `get_prediction_days` once.